# **0. Introduction**

For this challenge, you will be predicting a full year worth of sales for three items at two stores located in three different countries. your task is to predict the corresponding item sales for each date-country-store-item combination. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
ID = 'row_id'
TARGET = 'num_sold'
DATE = 'date'
YEAR = "year"
QUARTER = "quarter"
MONTH = "month"
WEEK = "week"
DAY = "day"
DAYOFYEAR = "dayofyear"
DAYOFMONTH = "dayofmonth"
DAYOFWEEK = "dayofweek"
DAY_NAME = "day_name"
MONTH_NAME = "month_name"

In [ ]:
def concat_df(train_data, test_data):
    # Returns a concatenated df of training and test set
    return pd.concat([train_data, test_data], sort=True).reset_index(drop=True)

def divide_df(all_data):
    # Returns divided dfs of training and test set
    return all_data.loc[:26297], all_data.loc[26298:].drop([TARGET], axis=1)

df_all = concat_df(train_df, test_df)

dfs = [train_df, test_df]



# **1. EDA**

In [ ]:
train_df.info()

In [ ]:
train_df[TARGET].describe()

In [ ]:
train_df.shape

In [ ]:
train_df.groupby('product').sum()

Kaggle Hat sold more than any other products

In [ ]:
train_df.groupby('store').sum()

Overall, KaggleRama store sold more products than KaggleMart

In [ ]:
train_df.groupby('country').sum()

Norway country store have highest sale.

**Observation:** Norway's Kaggle Rama stores sold more Kaggle Hat product than any other products during New year eve period. Let's confirm this observation via. data visualization later.

In [ ]:
date_wise_sales_count = train_df.groupby(DATE)[TARGET].sum().reset_index(name='num_of_products_sold')
date_wise_sales_count

In [ ]:
#top 10 highest sales date
date_wise_sales_count.sort_values(by='num_of_products_sold', ascending=False)[:10]

**Observation:** Overall in 4 years, around New year eve time, lot of products got sold than any other date in an year.

In [ ]:
train_df[DATE] =pd.to_datetime(train_df[DATE], format = '%Y-%m-%d')
test_df[DATE] =pd.to_datetime(test_df[DATE], format = '%Y-%m-%d')
#alernative approach - train_df = pd.read_csv(TRAIN_CSV, parse_dates=[DATE])

In [ ]:
train_df.info()

Let's create year and month feature to find the data distribution and see, whether we can find any meaningful insight from those features.

In [ ]:
def get_basic_ts_features(df):
    df[YEAR] = df[DATE].dt.year
    df[MONTH] = df[DATE].dt.month
    df[WEEK]= df[DATE].dt.week    
    df[DAY] = df[DATE].dt.day
    df[DAYOFWEEK] = df[DATE].dt.dayofweek
    df['is_weekend'] = (df[DAYOFWEEK]>=5).astype(int)    
    df[DAYOFMONTH] = df[DATE].dt.days_in_month
    df[DAYOFYEAR] = df[DATE].dt.dayofyear
    
    return df

In [ ]:
train_df = get_basic_ts_features(train_df)
test_df = get_basic_ts_features(test_df)

In [ ]:
train_df.head()

In [ ]:
countries = train_df['country'].unique()
stores = train_df['store'].unique()
countries = train_df['product'].unique()
years = train_df['year'].unique()

In [ ]:
train_df.info()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns

def create_line_plot(df):
    fig, ax = plt.subplots(figsize = (15, 7))

    chart = sns.lineplot(x='year', y=TARGET, hue="country",  ax=ax, data=df)

    ax.xaxis.set_major_locator(MaxNLocator(steps=[1]))

    plt.show()

In [ ]:
create_line_plot(train_df)

In Finland stores, every year, the sales is going up with constant confidence intervals. In Norway stores, in 2016, the sales went down and again increased in subsequent year. Sweden is consistent. 

In [ ]:
#sns.lineplot(data=train_df, x="date", y="num_sold") # so densed , unreadable
plt.figure(figsize = (15,8))
sns.lineplot(data=train_df, x="month", y="num_sold", hue="country")
plt.legend(loc='upper right')

the month-wise sales pattern is consistent across all the countries

In [ ]:
cat_features = ['store','product', 'country']

fig, axs = plt.subplots(ncols=1, nrows=3, figsize=(25,10))
#plt.subplots_adjust(right=1.5, top=1.25)

for i, feature in enumerate(cat_features, 1):
    plt.subplot(1, 3, i)
    sns.boxplot(x=feature, y="num_sold", notch=True , data=train_df)
    plt.xlabel('{}'.format(feature), size=10, labelpad=15) 
    plt.ylabel('Product Sold', size=10, labelpad=15) 
    #plt.yticks(np.arange(0,3000,1000))
    plt.ylim(0,1350)
    
plt.show()



* All categorical features distribution are **skewed right (i.e., positively skewed)**, due to large outliers
* Notches in the boxplots do not overlap, we can conclude that with **95% confidence**, that the true medians do differ.
* The whisker at the maximum is larger than other side, so this indicates more scattered data


In [ ]:
sns.barplot( y="num_sold", x="product", hue="store", data=train_df)

In [ ]:
sns.barplot( y="num_sold", x="country", hue="store", data=train_df)

sub_df['num_sold'].apply(np.ceil)https://www.officeholidays.com/countries/sweden
https://www.officeholidays.com/countries/finland
https://www.officeholidays.com/countries/norway

# **2. Feature Engineering**

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

encoded_features = []

for df in dfs:
    for feature in cat_features:
        encoded_feat = OneHotEncoder().fit_transform(df[feature].values.reshape(-1,1)).toarray()
        n = df[feature].nunique()
        cols = ['{}_{}'.format(feature, n) for n in range(1, n+1)]
        encoded_df = pd.DataFrame(encoded_feat, columns=cols)
        encoded_df.index = df.index
        encoded_features.append(encoded_df)
        
train_df = pd.concat([train_df, *encoded_features[:3]], axis=1)
test_df = pd.concat([test_df, *encoded_features[3:]], axis=1)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

# 3. Model

In [ ]:
# https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/36414
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 100*np.mean(diff)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
 
    return df

In [ ]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

In [ ]:
drop_cols = [ID, DATE]
drop_cols+=cat_features
features = [ col for col in train_df.columns if col not in drop_cols+[TARGET]]

In [ ]:
y=train_df[TARGET]
X=train_df.drop(columns=drop_cols+[TARGET])
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
from xgboost import XGBRegressor

xb_model = XGBRegressor()
xb_model.fit(X_train, y_train)

In [ ]:
print("XGBoost train data's mean accuracy = {} %".format(round((xb_model.score(X_train,y_train)*100),2)))

In [ ]:
y_pred_xb=xb_model.predict(X_test)
print("SMAPE={}".format(smape(y_test,y_pred_xb)))

In [ ]:
y_pred_test=xb_model.predict(test_df[features])

In [ ]:
y_pred_test

In [ ]:
sub_df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv')

sub_df[TARGET] = y_pred_test

sub_df[TARGET] = np.round(sub_df[TARGET]).astype(int)

sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv', index=False)